In [2]:
import numpy as np
import pandas as pd
# import plotly.plotly as py
import plotly.graph_objs as go 
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot as py
import matplotlib.pyplot as plt

from os import path

data_path = 'data'

In [2]:
init_notebook_mode(connected=True)

In [3]:
# List of SA countries
sa_countries = ['Argentina', 'Bolivia', 'Brazil', 'Chile', 'Colombia', 'Ecuador', 'Guyana', 'Paraguay', 'Peru', 'Uruguay', 'Venezuela, RB', 'Suriname']

In [4]:
def createWBDataFrame(path):
    df = pd.read_csv(path, header=2, encoding='utf-8')
    df = pd.merge(df, country_metadata, how='left', on=['Country Code'])
    return df.drop(['Indicator Name', 'Unnamed: 63'], axis=1)

def tideWBDataFrame(df):
    df = pd.melt(df, id_vars=['Country Name', 'Country Code', 'Region', 'Indicator Code'], var_name='Year', value_name='observation')
    df = pd.pivot_table(df, columns=['Indicator Code'], index=['Country Name', 'Country Code', 'Region', 'Year'], values='observation', aggfunc=np.mean)
    return df.reset_index()

def keepMetrics(df, metrics):
    standard_columns = ['Country Name', 'Country Code', 'Region', 'Year']
    return df.loc[:, standard_columns + metrics]

In [5]:
country_metadata = pd.read_csv(path.join(data_path, 'GDP_BY_COUNTRY', 'Metadata_Country_API_NY.GDP.MKTP.CD_DS2_en_csv_v2_10515210.csv'))
country_metadata.drop(['IncomeGroup', 'SpecialNotes', 'TableName', 'Unnamed: 5'], axis=1, inplace=True)

social_env_path = path.join(data_path, 'SOCIAL_ENV_ECON_FACTORS_BY_COUNTRY')

<h1><center>Extreme poverty</center></h1>

In [6]:
extreme_poverty_by_country = createWBDataFrame(path.join(social_env_path, 'Environment', 'API_19_DS2_en_csv_v2_10515758.csv'))
extreme_poverty_by_country = tideWBDataFrame(extreme_poverty_by_country)

# Limiting data for the last 30 years
extreme_poverty_by_country = extreme_poverty_by_country[(extreme_poverty_by_country['Year'] >= '2000') & (extreme_poverty_by_country['Year'] < '2018')]
extreme_poverty_by_country = keepMetrics(extreme_poverty_by_country, ['SI.POV.DDAY','SP.POP.TOTL'])

# Renaming observation columns
extreme_poverty_by_country.rename(columns={'SI.POV.DDAY': 'Extreme Poverty(% pop)','SP.POP.TOTL': 'Total Pop.'}, inplace=True)
extreme_poverty_by_country = extreme_poverty_by_country[extreme_poverty_by_country['Region'] == 'Latin America & Caribbean']
extreme_poverty_by_country = extreme_poverty_by_country[extreme_poverty_by_country['Country Name'].isin(sa_countries)]

In [7]:
# extreme_poverty_by_country.head()

## Education

In [8]:
edbc_path = path.join(social_env_path, 'Education', 'API_4_DS2_en_csv_v2_10577018.csv')
education_by_country = createWBDataFrame(edbc_path)
education_by_country = tideWBDataFrame(education_by_country)
education_by_country = keepMetrics(education_by_country, ['SE.PRM.CMPT.ZS', 
                                                          'SE.XPD.TOTL.GD.ZS',
                                                          'SE.XPD.PRIM.PC.ZS',
                                                          'SE.XPD.SECO.PC.ZS',
                                                          'SE.XPD.TERT.PC.ZS',
                                                          'SE.PRM.UNER.ZS',
                                                          'SE.PRM.ENRL.TC.ZS', 
                                                          'SE.ADT.1524.LT.ZS', 
                                                          ])

education_by_country.rename(columns={'SE.PRM.CMPT.ZS': 'Primary Completion Rate (% of relevant age group)', 
                                     'SE.XPD.PRIM.PC.ZS': 'Government expenditure per student, primary (% of GDP)',
                                     'SE.XPD.SECO.PC.ZS': 'Government expenditure per student, secondary (% of GDP)',
                                     'SE.XPD.TERT.PC.ZS': 'Government expenditure per student, tertiary (% of GDP)',
                                     'SE.PRM.UNER.ZS': 'Children out of School(% primary school)',  
                                     'SE.PRM.ENRL.TC.ZS': 'Pupil Teacher ratio', 
                                     'SE.ADT.1524.LT.ZS': 'Youth Literacy Rate 15-24 (% pop)', 
                                     'SE.XPD.TOTL.GD.ZS': 'Expenditure on Education (% of GDP)'}, inplace=True)
education_by_country = education_by_country[(education_by_country['Year'] >= '2000') & (education_by_country['Year'] < '2018')]

In [9]:
# education_by_country = education_by_country[education_by_country['Region'] == 'Latin America & Caribbean']
# education_by_country.head()

In [10]:
# education_by_country = education_by_country[education_by_country['Country Name'].isin(sa_countries)]
# education_by_country.info()

In [11]:
# education_by_country[education_by_country['Country Name'] == 'Brazil'].head()

In [12]:
selected_sa_countries = ['Argentina', 'Brazil', 'Chile', 'Colombia']
education_by_country_selected = education_by_country[education_by_country['Country Name'].isin(selected_sa_countries)]

In [13]:
layout = go.Layout( 
    xaxis = go.layout.XAxis(
        tickmode='array',
        tickvals=np.arange(2000, 2018, step=1),
        tickangle=45,
        title='Year',
        ticklen=5
    ),
    yaxis = go.layout.YAxis(
        ticksuffix='% ',
        range=[0, 7]
    ),
    title='Expenditure on Education (% of GDP), 2000-2017'
)

fig = go.Figure(
    layout=layout
)
        
for country in education_by_country_selected['Country Name'].unique():
    ext_pov_region = education_by_country_selected[education_by_country_selected['Country Name'] == country]
    x = ext_pov_region['Year']
    y = ext_pov_region['Expenditure on Education (% of GDP)'].fillna(method='ffill')
    fig.add_scatter(x=x, y=y, name=country, mode='lines')
    
py(fig, filename='gdp-edu-line')

In [14]:
def expenditure_edu_x_ext_poverty(country_name):
    edu_by_region = education_by_country_selected[education_by_country_selected['Country Name'] == country_name]
    ext_pov_by_region = extreme_poverty_by_country[extreme_poverty_by_country['Country Name'] == country_name]

    trace1 = go.Scatter(
        x=ext_pov_region['Year'],
        y=edu_by_region['Expenditure on Education (% of GDP)'].fillna(method='ffill'),
        name='Expenditure on Education',
    )
    
    trace2 = go.Scatter(
        x=ext_pov_region['Year'],
        y=ext_pov_by_region['Extreme Poverty(% pop)'].fillna(method='ffill'),
        name='Extreme Poverty',
        yaxis='y2'
    )

    data = [trace1, trace2]

    layout = go.Layout(
        title='Expenditure on Education x Extreme Poverty on {}'.format(country_name),
        yaxis=dict(
            title='% GDP'
        ),
        yaxis2=dict(
            title='% Population',
            overlaying='y',
            side='right'
        ),
        xaxis = go.layout.XAxis(
            title='Year',
            ticklen=5
        ),
    )
    
    fig = go.Figure(data=data, layout=layout)
    return fig

### Brazil

In [15]:
fig_brazil = expenditure_edu_x_ext_poverty('Brazil')
# py(fig_brazil)

### Argentina

In [16]:
fig_argentina = expenditure_edu_x_ext_poverty('Argentina')
# py(fig_argentina)

In [1]:
# fig = go.Figure(layout=go.Layout(xaxis=dict(showline=False)))

fig = tools.make_subplots(
        rows=2, 
        cols=2, 
        subplot_titles=[str(c) for c in education_by_country_selected['Country Name'].unique()],
        vertical_spacing=0.25,
        horizontal_spacing=0.20)

exp_education = []
ext_pov = []

for country_name in education_by_country_selected['Country Name'].unique():
    
    edu_by_region = education_by_country_selected[education_by_country_selected['Country Name'] == country_name]
    ext_pov_by_region = extreme_poverty_by_country[extreme_poverty_by_country['Country Name'] == country_name]
   
    trace1 = go.Scatter(
        x=ext_pov_region['Year'],
        y=ext_pov_by_region['Extreme Poverty(% pop)'].fillna(method='ffill'),
        name='Extreme Poverty',
        mode='lines',
        showlegend=False,
        marker=go.scatter.Marker(color='darkorange'),
        yaxis='y2',
        hoverinfo='y',
    )
    
    trace2 = go.Scatter(
        x=ext_pov_region['Year'],
        y=edu_by_region['Expenditure on Education (% of GDP)'].fillna(method='ffill'),
        name='Expenditure on Education',
        mode='lines',
        showlegend=False,
        marker=go.scatter.Marker(color='Blue'),
        hoverinfo='y'
    )

    
    exp_education.append(trace1)
    ext_pov.append(trace2)

for i in range(4):
    col = i % 2 + 1
    row = i // 2 + 1
    
    exp_education[0]['showlegend'] = True
    ext_pov[0]['showlegend'] = True
    
    if i < 4:
        fig.append_trace(exp_education[i], row, col)
        fig.append_trace(ext_pov[i], row, col)
        
for i in range(4):
    fig['data'][2*i+1].update(yaxis='y{one}'.format(one=5+i)) 

fig['layout'].update(go.Layout(
                        title='Expenditure on Education by Extreme Poverty',
                        height=600,
                        width=1000,
    
                        xaxis1 = go.layout.XAxis(title='Year', tickmode='array', tickvals=np.arange(2000, 2018, step=5), ticklen=5),
                        xaxis2 = go.layout.XAxis(title='Year', tickmode='array', tickvals=np.arange(2000, 2018, step=5), ticklen=5),
                        xaxis3 = go.layout.XAxis(title='Year', tickmode='array', tickvals=np.arange(2000, 2018, step=5), ticklen=5),
                        xaxis4 = go.layout.XAxis(title='Year', tickmode='array', tickvals=np.arange(2000, 2018, step=5), ticklen=5),
                        
                        yaxis1 = go.layout.YAxis(title='Pop. in Extreme Poverty (%)', range=[0, 15]),
                        yaxis2 = go.layout.YAxis(title='Pop. in Extreme Poverty (%)', range=[0, 30]),
                        yaxis3 = go.layout.YAxis(title='Pop. in Extreme Poverty (%)', range=[0, 15]),
                        yaxis4 = go.layout.YAxis(title='Pop. in Extreme Poverty (%)', range=[0, 20]),
                        
                        yaxis5 = go.layout.YAxis(title='GDP (%)', overlaying='y1', anchor='x1', side='right', showgrid=False, range=[0,8]),
                        yaxis6 = go.layout.YAxis(title='GDP (%)', overlaying='y2', anchor='x2', side='right', showgrid=False, range=[0,8]),
                        yaxis7 = go.layout.YAxis(title='GDP (%)', overlaying='y3', anchor='x3', side='right', showgrid=False, range=[0,8]),
                        yaxis8 = go.layout.YAxis(title='GDP (%)', overlaying='y4', anchor='x4', side='right', showgrid=False, range=[0,8]),
                        
                        )
                    )

py(fig)

NameError: name 'tools' is not defined

## Resources distribuition on Brazil

In [18]:
x_years = ['2009', '2011', '2013', '2015']
education_by_country_selected = education_by_country_selected[education_by_country_selected['Year'].isin(x_years)]

In [19]:
# years = 2011, 2013, 2015
resources_dist_brazil = education_by_country[education_by_country['Country Name'] == 'Brazil']
resources_dist_brazil = resources_dist_brazil[(resources_dist_brazil['Year'] == '2009') | (resources_dist_brazil['Year'] == '2011') | (resources_dist_brazil['Year'] == '2013') | (resources_dist_brazil['Year'] == '2015')]
# resources_dist_brazil.head()

In [20]:
# fig = go.Figure(layout=go.Layout(xaxis=dict(showline=False)))

fig = tools.make_subplots(
        rows=2, 
        cols=2, 
        subplot_titles=[str(c) for c in education_by_country_selected['Country Name'].unique()],
        vertical_spacing=0.25,
        horizontal_spacing=0.20)

primary = []
secondary = []
tertiary = []

for country_name in education_by_country_selected['Country Name'].unique():
    
    edu_by_region = education_by_country_selected[education_by_country_selected['Country Name'] == country_name]
    
    
    trace1 = go.Bar(
        x=x_years,
        y=edu_by_region['Government expenditure per student, primary (% of GDP)'],
        name='Primary',
        showlegend=False,
        marker=go.bar.Marker(color='Blue'),
    )
    
    trace2 = go.Bar(
        x=x_years,
        y=edu_by_region['Government expenditure per student, secondary (% of GDP)'],
        name='Secondary',
        showlegend=False,
        marker=go.bar.Marker(color='darkorange'),
    )

    trace3 = go.Bar(
        x=x_years,
        y=edu_by_region['Government expenditure per student, tertiary (% of GDP)'],
        name='Tertiary',
        showlegend=False,
        marker=go.bar.Marker(color='Green'),

    )
    
    primary.append(trace1)
    secondary.append(trace2)
    tertiary.append(trace3)

for i in range(4):
    col = i % 2 + 1
    row = i // 2 + 1
    
    primary[0]['showlegend'] = True
    secondary[0]['showlegend'] = True
    tertiary[0]['showlegend'] = True
    
    if i < 4:
        fig.append_trace(primary[i], row, col)
        fig.append_trace(secondary[i], row, col)
        fig.append_trace(tertiary[i], row, col)
        
# for i in range(4):
#     fig['data'][2*i+1].update(yaxis='y{one}'.format(one=5+i)) 

fig['layout'].update(go.Layout(
                        title='Expenditure on Education by Extreme Poverty',
                        height=600,
                        width=1000,
    
                        xaxis1 = go.layout.XAxis(title='Year', tickmode='array', ticklen=5),
                        xaxis2 = go.layout.XAxis(title='Year', tickmode='array', ticklen=5),
                        xaxis3 = go.layout.XAxis(title='Year', tickmode='array', ticklen=5),
                        xaxis4 = go.layout.XAxis(title='Year', tickmode='array', ticklen=5),
                        
                        yaxis1 = go.layout.YAxis(title='GDP (%)', range=[0, 35]),
                        yaxis2 = go.layout.YAxis(title='GDP (%)', range=[0, 35]),
                        yaxis3 = go.layout.YAxis(title='GDP (%)', range=[0, 35]),
                        yaxis4 = go.layout.YAxis(title='GDP (%)', range=[0, 35]),
                        
#                         yaxis5 = go.layout.YAxis(title='GDP (%)', overlaying='y1', anchor='x1', side='right', showgrid=False, range=[0,8]),
#                         yaxis6 = go.layout.YAxis(title='GDP (%)', overlaying='y2', anchor='x2', side='right', showgrid=False, range=[0,8]),
#                         yaxis7 = go.layout.YAxis(title='GDP (%)', overlaying='y3', anchor='x3', side='right', showgrid=False, range=[0,8]),
#                         yaxis8 = go.layout.YAxis(title='GDP (%)', overlaying='y4', anchor='x4', side='right', showgrid=False, range=[0,8]),
                        
                        )
                    )

py(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3 ]  [ (2,2) x4,y4 ]



In [21]:
# x_years = ['2009', '2011', '2013', '2015']

# trace1 = go.Bar(
#     x=x_years,
#     y=resources_dist_brazil['Government expenditure per student, primary (% of GDP)'],
#     name='Primary',
# )
# trace2 = go.Bar(
#     x=x_years,
#     y=resources_dist_brazil['Government expenditure per student, secondary (% of GDP)'],
#     name='Secondary',
# )

# trace3 = go.Bar(
#     x=x_years,
#     y=resources_dist_brazil['Government expenditure per student, tertiary (% of GDP)'],
#     name='Tertiary',
# )

# data = [trace1, trace2, trace3]
# layout = go.Layout(
#     title='Government expenditure per student by schollarity level on Brazil',
#     barmode='group',
#     yaxis = go.layout.YAxis(
#         title='GDP (%)',
#         ticksuffix='% '
#     ),
#     xaxis = go.layout.XAxis(
#         title='Year'
#     ),
# )

# fig = go.Figure(data=data, layout=layout)
# py(fig, filename='resources-dist-brazil-edu')

# Climate change

In [22]:
env_path = path.join(social_env_path, 'Environment', 'API_19_DS2_en_csv_v2_10515758.csv')
access_to_eletricity_by_country = createWBDataFrame(env_path)
access_to_eletricity_by_country = tideWBDataFrame(access_to_eletricity_by_country)

# Limiting data for the last 30 years
access_to_eletricity_by_country = access_to_eletricity_by_country[access_to_eletricity_by_country['Year'] > '1980']
access_to_eletricity_by_country = keepMetrics(access_to_eletricity_by_country, ['EG.ELC.ACCS.ZS', 'SH.DYN.MORT'])

# Renaming observation columns
access_to_eletricity_by_country.rename(columns={'EG.ELC.ACCS.ZS': 'Access to electricity (% of pop)',
                                                'SH.DYN.MORT': 'Mortality rate, under-5 (per 1,000 live births)'
                                               }, inplace=True)

access_to_eletricity_by_country = access_to_eletricity_by_country[(access_to_eletricity_by_country['Year'] >= '2000') & (access_to_eletricity_by_country['Year'] < '2018')]
access_to_eletricity_by_country = access_to_eletricity_by_country[access_to_eletricity_by_country['Region'] == 'Latin America & Caribbean']
access_to_eletricity_by_country = access_to_eletricity_by_country[access_to_eletricity_by_country['Country Name'].isin(selected_sa_countries)]

In [23]:
# access_to_eletricity_by_country.head()

In [24]:
access_by_region = access_to_eletricity_by_country[access_to_eletricity_by_country['Country Name'] == 'Chile']
ext_pov_by_region = extreme_poverty_by_country[extreme_poverty_by_country['Country Name'] == 'Chile']

trace1 = go.Scatter(
    x=ext_pov_region['Year'],
    y=access_by_region['Access to electricity (% of pop)'].fillna(method='ffill'),
    name='Access to electricity'
)
trace2 = go.Scatter(
    x=ext_pov_region['Year'],
    y=ext_pov_by_region['Extreme Poverty(% pop)'].fillna(method='ffill'),
    name='Extreme Poverty',
    yaxis='y2'
)

data = [trace1, trace2]

layout = go.Layout(
    title='Access to electricity x Extreme Poverty on Brazil',
    yaxis=dict(
        title='% Population'
    ),
    yaxis2=dict(
        title='% Population',
        overlaying='y',
        side='right'
    ),
    xaxis = go.layout.XAxis(
        title='Year',
    ),
)
fig_brazil = go.Figure(data=data, layout=layout)
# py(fig_brazil, filename='gdp-edu-line')

# Mortality rate

In [25]:
layout = go.Layout( 
    xaxis = go.layout.XAxis(
        tickmode='array',
        tickvals=np.arange(1990, 2018, step=1),
        tickangle=45,
        title='Year',
        ticklen=5
    ),
    yaxis = go.layout.YAxis(
        range=[0, 40],
        ticksuffix='% ',
#         title='Mortality rate (%)'
    ),
    title='Mortality rate, under-5 (per 1,000 live births), 2000-2017'
)

fig = go.Figure(
    layout=layout
)
        
for country in access_to_eletricity_by_country['Country Name'].unique():
    ate_region = access_to_eletricity_by_country[access_to_eletricity_by_country['Country Name'] == country]
    x = ate_region['Year']
    y = ate_region['Mortality rate, under-5 (per 1,000 live births)'].fillna(method='ffill')
    fig.add_scatter(x=x, y=y, name=country, mode='lines')
    
py(fig, filename='gdp-edu-line')

In [26]:
# access_by_region = access_to_eletricity_by_country[access_to_eletricity_by_country['Country Name'] == 'Brazil']
# ext_pov_by_region = extreme_poverty_by_country[extreme_poverty_by_country['Country Name'] == 'Brazil']

# trace1 = go.Scatter(
#     x=ext_pov_region['Year'],
#     y=access_by_region['Mortality rate, under-5 (per 1,000 live births)'].fillna(method='ffill'),
#     name='Mortality Rate'
# )
# trace2 = go.Scatter(
#     x=ext_pov_region['Year'],
#     y=ext_pov_by_region['Extreme Poverty(% pop)'].fillna(method='ffill'),
#     name='Extreme Poverty',
#     yaxis='y2'
# )

# data = [trace1, trace2]

# layout = go.Layout(
#     title='Mortality rate (under-5) x Extreme Poverty on Brazil',
#     yaxis=dict(
#         title='(under-5)'
#     ),
#     yaxis2=dict(
#         title='% Population',
#         overlaying='y',
#         side='right'
#     ),
#     xaxis = go.layout.XAxis(
#         title='Year',
#     ),
# )
# fig_brazil = go.Figure(data=data, layout=layout)
# py(fig_brazil, filename='gdp-edu-line')

In [30]:
fig = tools.make_subplots(
        rows=2, 
        cols=2, 
        subplot_titles=[str(c) for c in access_to_eletricity_by_country['Country Name'].unique()],
        vertical_spacing=0.25,
        horizontal_spacing=0.20)

mortality_rate = []
ext_pov = []

for country_name in access_to_eletricity_by_country['Country Name'].unique():
    
    mortal_by_region = access_to_eletricity_by_country[access_to_eletricity_by_country['Country Name'] == country_name]
    ext_pov_by_region = extreme_poverty_by_country[extreme_poverty_by_country['Country Name'] == country_name]
    
    trace1 = go.Scatter(
        x=ext_pov_region['Year'],
        y=ext_pov_by_region['Extreme Poverty(% pop)'].fillna(method='ffill'),
        name='Extreme Poverty',
        mode='lines',
        showlegend=False,
        marker=go.scatter.Marker(color='darkorange'),
        yaxis='y2',
        hoverinfo='y',
    )

    trace2 = go.Scatter(
        x=ext_pov_region['Year'],
        y=mortal_by_region['Mortality rate, under-5 (per 1,000 live births)'].fillna(method='ffill'),
        name='Mortality rate',
        mode='lines',
        showlegend=False,
        marker=go.scatter.Marker(color='Blue'),
        hoverinfo='y'
    )
    
    mortality_rate.append(trace1)
    ext_pov.append(trace2)

for i in range(4):
    col = i % 2 + 1
    row = i // 2 + 1
    
    mortality_rate[0]['showlegend'] = True
    ext_pov[0]['showlegend'] = True

    fig.append_trace(mortality_rate[i], row, col)
    fig.append_trace(ext_pov[i], row, col)
        
for i in range(4):
    fig['data'][2*i+1].update(yaxis='y{one}'.format(one=5+i)) 

fig['layout'].update(go.Layout(
                        title=go.layout.Title(text='Mortality rate, under-5 (per 1,000 live births) by Extreme Poverty'),
                        height=600,
                        width=1000,
    
                        xaxis1 = go.layout.XAxis(title='Year', tickmode='array', tickvals=np.arange(2000, 2018, step=5), ticklen=5),
                        xaxis2 = go.layout.XAxis(title='Year', tickmode='array', tickvals=np.arange(2000, 2018, step=5), ticklen=5),
                        xaxis3 = go.layout.XAxis(title='Year', tickmode='array', tickvals=np.arange(2000, 2018, step=5), ticklen=5),
                        xaxis4 = go.layout.XAxis(title='Year', tickmode='array', tickvals=np.arange(2000, 2018, step=5), ticklen=5),
                        
                        yaxis1 = go.layout.YAxis(title='Pop. in Extreme Poverty (%)', range=[0, 15]),
                        yaxis2 = go.layout.YAxis(title='Pop. in Extreme Poverty', range=[0, 30]),
                        yaxis3 = go.layout.YAxis(title='Pop. in Extreme Poverty', range=[0, 15]),
                        yaxis4 = go.layout.YAxis(title='Pop. in Extreme Poverty', range=[0, 20]),
                        
                        yaxis5 = go.layout.YAxis(title='Mortality rate (%)', overlaying='y1', anchor='x1', side='right', showgrid=False, range=[0, 20]),
                        yaxis6 = go.layout.YAxis(title='Mortality rate (%)', overlaying='y2', anchor='x2', side='right', showgrid=False, range=[0, 40]),
                        yaxis7 = go.layout.YAxis(title='Mortality rate (%)', overlaying='y3', anchor='x3', side='right', showgrid=False, range=[0, 15]),
                        yaxis8 = go.layout.YAxis(title='Mortality rate (%)', overlaying='y4', anchor='x4', side='right', showgrid=False, range=[0, 40]),
                        
                        )
                    )

py(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3 ]  [ (2,2) x4,y4 ]

